In [2]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
import tempfile
import os
from joblib import load, dump
import shutil

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
testset = datasets.MNIST('./data', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = len(trainset))
testloader = torch.utils.data.DataLoader(testset, batch_size = len(testset))
trainset_array1 = next(iter(trainloader))[0].numpy()
testset_array1 = next(iter(testloader))[0].numpy()

trainset_labels_array1 = next(iter(trainloader))[1].numpy()
testset_labels_array1 = next(iter(testloader))[1].numpy()

trainset_array = trainset_array1.reshape(60000,784)
testset_array = testset_array1.reshape(10000,784)


In [2]:
temp_folder_test = tempfile.mkdtemp()
filename_test = os.path.join(temp_folder_test, 'test.mmap')
if os.path.exists(filename_test): os.unlink(filename_test)
_ = dump(testset_array, filename_test)
test_memmap = load(filename_test, mmap_mode='r+')

temp_folder_train = tempfile.mkdtemp()
filename_train = os.path.join(temp_folder_train, 'train.mmap')
if os.path.exists(filename_train): os.unlink(filename_train)
_ = dump(trainset_array, filename_train)
train_memmap = load(filename_train, mmap_mode='r+')

del testset_array
del trainset_array
import gc
_ = gc.collect()

test_memmap_small = test_memmap[:512]
train_memmap_small = train_memmap[:512]

test_memmap_final = np.asarray(test_memmap_small)
train_memmap_final = np.asarray(train_memmap_small)

In [3]:
def calc_euclid(x,y):
    dist = sum((x-y)**2)
    return dist

In [4]:
testset_array2 = testset_array[:512]
trainset_array2 = trainset_array[:512]

In [6]:
print('task started')
start = datetime.now()
final_list = Parallel(n_jobs=8,backend='multiprocessing',batch_size=32768)(delayed(calc_euclid)(x,y) for x in testset_array2 for y in trainset_array2)
# final_list1 = np.asarray(final_list)
# final_list1 = np.split(final_list1,512)
n=512
final = [final_list[i:i + n] for i in range(0, len(final_list), n)]  
l1 = []
i=0
for x in final:
    y = x.index(min(x))
    l1.append(trainset_labels_array1[y])
    i+=1
end = datetime.now()
time_taken = end - start
print('Time: ',time_taken)
print(len(l1))

# try:
#     shutil.rmtree(temp_folder_test)
#     shutil.rmtree(temp_folder_train)
# except OSError:
#     pass

task started
Time:  0:00:28.244859
512


In [7]:
l1 = np.asarray(l1)
l1

array([7, 2, 1, 0, 9, 1, 9, 9, 4, 9, 0, 6, 9, 0, 1, 3, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 1,
       5, 5, 1, 8, 4, 4, 6, 3, 5, 5, 0, 0, 4, 1, 9, 1, 7, 2, 4, 9, 7, 4,
       0, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 3, 9, 7, 9, 6, 2, 9, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 1, 5, 9, 2, 9, 4, 4, 9, 7, 3, 7, 9, 6, 9, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 9, 2, 0, 1, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 1, 4, 6, 5, 4, 4, 8, 4, 5, 1, 9, 4, 7, 2, 3, 2, 1,
       1, 9, 1, 8, 1, 2, 5, 0, 3, 9, 2, 3, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       9, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 9, 9, 7, 9, 0, 3, 5, 7, 5,
       7, 3, 2, 7, 1, 2, 8, 4, 1, 7, 3, 5, 5, 9, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 4, 7, 5, 0, 6, 4, 1, 4, 1, 9, 5, 7, 9, 2, 1, 2, 6, 8, 1, 7, 7,
       4, 1, 0, 1, 3, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 1, 9, 2,
       6, 4, 1, 8, 4, 2, 9, 2, 0, 9, 0, 0, 2, 8, 1,

In [ ]:
final_list1 = np.split(final_list,512)

In [ ]:
len(final_list1)

In [ ]:
final_list1 = np.asarray(final_list1)

In [ ]:
l1 = []
i=0
for x in final_list1:
    y = np.argmin(x)
    l1.append(trainset_labels_array1[y])
    i+=1

In [ ]:
l1 = np.asarray(l1)
l1.shape

In [ ]:
l2 = []
j = 0
for j in range(0,512):
    l2.append(testset_labels_array1[j])
l2= np.asarray(l2)
l2.shape

In [ ]:
j = 0
for x in range(0,512):
    if l1[x] == l2[x]:
        j += 1
j